In [ ]:
import org.apache.lucene.analysis.standard.StandardAnalyzer;
import org.apache.lucene.document.*;
import org.apache.lucene.index.*;
import org.apache.lucene.search.*;
import org.apache.lucene.search.knn.KnnVectorQuery;
import org.apache.lucene.store.FSDirectory;

import java.io.IOException;
import java.nio.file.Paths;
import java.util.List;

public class LuceneMovieSearch {

    static class Movie {
        String id;
        String title;
        int year;
        float rating;
        float[] embedding;
        Movie(String id, String title, int year, float rating, float[] embedding) {
            this.id = id;
            this.title = title;
            this.year = year;
            this.rating = rating;
            this.embedding = embedding;
        }
    }

    public static void main(String[] args) throws IOException {
        String indexPath = "movie_index";
        FSDirectory dir = FSDirectory.open(Paths.get(indexPath));

        StandardAnalyzer analyzer = new StandardAnalyzer();
        IndexWriterConfig config = new IndexWriterConfig(analyzer);
        config.setOpenMode(IndexWriterConfig.OpenMode.CREATE);
        IndexWriter writer = new IndexWriter(dir, config);

        // 1. CREATE INDEX & INSERT MOVIES
        List<Movie> movies = List.of(
                new Movie("1", "The Matrix", 1999, 8.7f, new float[]{0.1f, 0.3f, 0.5f}),
                new Movie("2", "Inception", 2010, 8.8f, new float[]{0.2f, 0.1f, 0.4f}),
                new Movie("3", "Pulp Fiction", 1994, 8.9f, new float[]{0.05f, 0.2f, 0.45f})
        );

        for (Movie m : movies) {
            Document doc = new Document();
            doc.add(new StringField("id", m.id, Field.Store.YES));
            doc.add(new TextField("title", m.title, Field.Store.YES));
            doc.add(new IntPoint("year", m.year));
            doc.add(new StoredField("year", m.year));
            doc.add(new FloatPoint("rating", m.rating));
            doc.add(new StoredField("rating", m.rating));
            doc.add(new KnnVectorField("embedding", m.embedding)); // vector field
            writer.addDocument(doc);
        }
        writer.close();

        // 2. QUERY (vector similarity + filter)
        DirectoryReader reader = DirectoryReader.open(dir);
        IndexSearcher searcher = new IndexSearcher(reader);

        float[] queryVector = new float[]{0.1f, 0.25f, 0.45f};
        Query vectorQuery = new KnnVectorQuery("embedding", queryVector, 5);
        Query yearFilter = IntPoint.newRangeQuery("year", Integer.MIN_VALUE, 1999);

        BooleanQuery combinedQuery = new BooleanQuery.Builder()
                .add(vectorQuery, BooleanClause.Occur.MUST)
                .add(yearFilter, BooleanClause.Occur.FILTER)
                .build();

        TopDocs topDocs = searcher.search(combinedQuery, 5);

        System.out.println("Top results (year < 2000):");
        for (ScoreDoc sd : topDocs.scoreDocs) {
            Document d = searcher.doc(sd.doc);
            System.out.printf("- %s (%s) | Score: %.4f%n", d.get("title"), d.get("year"), sd.score);
        }

        reader.close();
        dir.close();
    }
}
